### 💻 **<i>Notebook 02:</i> RAG Chain Development and LangSmith Tracing**

**For running on Google Colab only, run the code cell below:**

In [ ]:
import os
from dotenv import load_dotenv
try:
    # This import ONLY works on the Colab web interface
    from google.colab import userdata

    print("Loading API keys from Colab Secrets...")

    # --- Key Retrieval and Injection into os.environ ---
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_CSE_ID'] = userdata.get('GOOGLE_CSE_ID')
    os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
    os.environ['LANGCHAIN_PROJECT'] = "retrogaming-qa-bot"

    # --- Verification ---
    if not os.environ.get('OPENAI_API_KEY'):
        raise ValueError("OPENAI_API_KEY was not found in Colab Secrets.")

    print("✅ All API keys were successfully loaded from Colab Secrets.")

except ImportError:
    # Fallback for local environments (local VS Code)
    print("Loading API keys from local .env file (VS Code/Local Mode).")
    load_dotenv()

# --- Common Configurations ---
CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

# Here, the rest of the notebook variables are defined (llm, embeddings, etc.)
# ... (should continue defining LLM/RAG configuration variables)

A carregar chaves API do Colab Secrets...
✅ Todas as chaves API foram carregadas com sucesso a partir do Colab Secrets.


##### **00 - Configuration, Imports, and LangSmith Setup**

In [ ]:
pip install langchain_openai langchain_community chromadb langsmith python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 9.9 MB/s eta 0:00:00

In [ ]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# --- CRITICAL FIX: The chains have moved to a dedicated package ---
try:
    from langchain.chains.combine_documents import create_stuff_documents_chain
    from langchain.chains import create_retrieval_chain
    print("✅ Using modern LangChain libraries for Chains.")
except ImportError:
    # Fallback to the classic packages if the user environment is older
    print("⚠️ 'langchain.chains' not found. Attempting to use 'langchain_classic'...")
    from langchain_classic.chains.combine_documents import create_stuff_documents_chain
    from langchain_classic.chains import create_retrieval_chain


# --- 00 - Configuration, Imports, and LangSmith Setup ---
load_dotenv()

CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

# Configure LangSmith (optional, but recommended for tracing)
if os.getenv("LANGCHAIN_API_KEY"):
     os.environ["LANGCHAIN_TRACING_V2"] = "true"
     os.environ["LANGCHAIN_PROJECT"] = "retrogaming-qa-bot"
     print("✅ Configuration loaded. LangSmith tracing is enabled.")
else:
     print("✅ Configuration loaded. LangSmith tracing is DISABLED (set LANGCHAIN_API_KEY in .env to enable).")

# 1. Initialize LLM and Embeddings
llm = ChatOpenAI(model=LLM_MODEL, temperature=0.0)
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

# 2. Load the existing vector store
print(f"\n[1/3] Loading Vector Store from: {CHROMA_PATH}...")
vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
print("✅ Vector Store loaded.")

⚠️ 'langchain.chains' not found. Attempting to use 'langchain_classic'...
✅ Configuration loaded. LangSmith tracing is enabled.

[1/3] Loading Vector Store from: ./chroma_db_retrogaming...


/tmp/ipython-input-2809816407.py:42: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)


✅ Vector Store loaded.


##### **01 - RAG Chain**

In [ ]:
# --- 01 - RAG Chain Development ---

# 3. Define the Retriever
# The retriever is set to fetch the top 3 most relevant documents
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# 4. Define the Prompt for Generation
# This prompt is used to instruct the LLM on how to use the returned context
# and how to format its final answer.
SYSTEM_PROMPT_TEMPLATE = (
    "You are an expert retrogaming assistant and tuning specialist. Your expertise is in optimizing "
    "old games for better resolution and modern compatibility on PC/Mac or native consoles. "
    "Provide your answers concisely and professionally. "
    "Use ONLY the provided context below to answer the question. "
    "If the context does not contain the answer, state that you do not have enough information in the database. "
    "Do not invent or use external knowledge (the RAG tool is your only source here).\n\n"
    "Context:\n"
    "{context}\n\n"
    "Question: {input}"
)
rag_prompt = ChatPromptTemplate.from_template(SYSTEM_PROMPT_TEMPLATE)

# 5. Create the RAG Chain
# document_chain: Combines the retrieved documents with the prompt for the LLM
document_chain = create_stuff_documents_chain(llm, rag_prompt)

# rag_chain: The full chain that first retrieves documents and then passes them to the document_chain
rag_chain = create_retrieval_chain(retriever, document_chain)
print("\n[2/3] ✅ RAG Chain (Retrieval and Generation) assembled.")


[2/3] ✅ RAG Chain (Retrieval and Generation) assembled.


##### **02 - Testing**

In [ ]:
# --- 03 - Testing ---

# 6. Test the RAG Chain
question = "What setting should I check in a classic game to fix a stretched resolution issue?"

print(f"\n[3/3] ❓ Asking test question: {question}")
response = rag_chain.invoke({"input": question})

# Print the final answer
print("\n🤖 Final Answer:")
print(response["answer"])

# Print the sources used (now includes 'source_type' from Notebook 01)
print("\n🔗 Sources Used:")
for doc in response["context"]:
    source_type = doc.metadata.get('source_type', 'N/A')
    source = doc.metadata.get('source', 'N/A')
    print(f"- Type: {source_type.upper()}, Source: {source}")


print("\nNotebook 02 complete. The RAG Chain is functional with unified Web and YouTube data.")


[3/3] ❓ Asking test question: What setting should I check in a classic game to fix a stretched resolution issue?



🤖 Final Answer:
You should check the aspect ratio setting in the game options menu to fix a stretched resolution issue.

🔗 Sources Used:

Notebook 02 complete. The RAG Chain is functional with unified Web and YouTube data.
